In [1]:
# ResNet-v2的简单实现,参考TensorFlow实战

In [2]:
import collections
import tensorflow as tf
slim = tf.contrib.slim

/home/mzx/.local/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# 采用collections.namedtuple设计ResNet的基本Block模块
class Block(collections.namedtuple('Block', ['scope','unit_fn','args'])):
    'A named tuple describing a ResNet block.'

In [4]:
def subsample(inputs, factor, scope=None):
    """The subsample function for the bottlenecks"""
    if factor == 1:
        return inputs
    else:
        return slim.max_pool2d(inputs,[1,1],stride=factor,scope=scope)

In [5]:
def conv2d_same(inputs, num_outputs, kernel_size, stride, scope=None):
    if stride == 1:
        return slim.conv2d(inputs,num_outputs,kernel_size,stride=1,
                           padding='SAME', scope=scope)
    else:
        pad_total = kernel_size - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        inputs = tf.pad(inputs, [[0,0], [pad_beg, pad_end],
                                 [pad_beg, pad_end],[0,0]])
        return slim.conv2d(inputs,num_outputs,kernel_size,stride=stride,
                           padding='VALID', scope=scope)

In [6]:
@slim.add_arg_scope
def stack_blocks_dense(net,blocks,outputs_collections=None):
    for block in blocks:
        with tf.variable_scope(block.scope,'block',[net]) as sc:
            for i, unit in enumerate(block.args):
                with tf.variable_scope('unit_%d' % (i+1), values=[net]):
                    unit_depth,unit_depth_bottleneck,unit_stride = unit
                    net = block.unit_fn(net,
                                        depth=unit_depth,
                                        depth_bottleneck=unit_depth_bottleneck,
                                        stride=unit_stride)
                net = slim.utils.collect_named_outputs(outputs_collections,sc.name,net)
    return net

In [7]:
def resnet_arg_scope(is_training=True,
                      weight_decay=0.0001,
                      batch_norm_decay=0.997,
                      batch_norm_epsilon=1e-5,
                      batch_norm_scale=True):
    batch_norm_params = {"is_training": is_training,
                         "decay": weight_decay,
                         "epsilon": batch_norm_epsilon,
                         "scale": batch_norm_scale,
                         "updates_collections": tf.GraphKeys.UPDATE_OPS,
                        }
    with slim.arg_scope([slim.conv2d], weights_regularizer=slim.l2_regularizer(weight_decay),
                        weights_initializer=slim.variance_scaling_initializer(),
                        activation_fn=tf.nn.relu,
                        normalizer_fn=slim.batch_norm,
                        normalizer_params=batch_norm_params):
        with slim.arg_scope([slim.batch_norm], **batch_norm_params):
            with slim.arg_scope([slim.max_pool2d], padding='SAME') as arg_sc:
                return arg_sc

In [8]:
@slim.add_arg_scope
def bottleneck(inputs, depth, depth_bottleneck, stride,
               outputs_collections=None, scope=None):
    with tf.variable_scope(scope, 'bottleneck_v2',[inputs]) as sc:
        depth_in = slim.utils.last_dimension(inputs.get_shape(),min_rank=4)
        preact = slim.batch_norm(inputs, activation_fn=tf.nn.relu, scope='preact')
        if depth == depth_in:
            shortcut = subsample(inputs,stride,'shortcut')
        else:
            shortcut = slim.conv2d(preact, depth, [1, 1], stride=1, 
                                   normalizer_fn=None, activation_fn=None,
                                   scope='shortcut')
        
        residual = slim.conv2d(preact, depth_bottleneck, [1,1], stride=1,
                               scope='conv1')
        residual = conv2d_same(residual, depth_bottleneck, 3, stride,
                               scope='conv2')
        residual = slim.conv2d(residual, depth, [1,1], stride=1,
                               normalizer_fn=None, activation_fn=None,
                               scope='conv3')
        output = shortcut + residual
        
        return slim.utils.collect_named_outputs(outputs_collections,
                                                sc.name, output)

In [9]:
def resnet_v2(inputs, blocks, num_classes=None,
              global_pool=True, include_root_block=True,
              reuse=None, scope=None):
    with tf.variable_scope(scope,'resnet_v2',[inputs],reuse=reuse) as sc:
        end_points_collection = sc.original_name_scope + '_end_points'
        with slim.arg_scope([slim.conv2d,bottleneck,
                             stack_blocks_dense],
                            outputs_collections=end_points_collection):
            net = inputs
            if include_root_block:
                with slim.arg_scope([slim.conv2d], activation_fn=None,
                                    normalizer_fn=None):
                    net=conv2d_same(net,64,7,stride=2,scope='conv1')
                net=slim.max_pool2d(net,[3,3],stride=2,scope='conv1')
        net=stack_blocks_dense(net,blocks)
        net=slim.batch_norm(net, activation_fn=tf.nn.relu,
                            scope='postnorm')
        if global_pool:
            net=tf.reduce_mean(net,[1,2],name='pool5',keep_dims=True)
        if num_classes is not None:
            net=slim.conv2d(net, num_classes, [1,1],
                            activation_fn=None,
                            normalizer_fn=None,
                            scope='logits')
        end_points = slim.utils.convert_collection_to_dict(
                        end_points_collection)
        if num_classes is not None:
            end_points['predictions']=slim.softmax(net, scope='predictions')
    return net, end_points

In [10]:
def resnet_v2_152(inputs, num_classes=None,
                  global_pool=True, reuse=None,
                  scope='resnet_v2_152'):
    blocks= [
        Block('block1', bottleneck, [(256,64,1)]*2 + [(256,64,2)]),
        Block('block2', bottleneck, [(512,128,1)]*7 + [(512,128,2)]),
        Block('block3', bottleneck, [(1024,256,1)]*35 + [(1024,256,2)]),
        Block('block4', bottleneck, [(2048,512,1)]*3)
    ]
    return resnet_v2(inputs, blocks, num_classes, global_pool,
                     include_root_block=True,
                     reuse=reuse,
                     scope=scope)

In [11]:
import math
import time
from datetime import datetime
def time_tensorflow_run(session, target, info_string, num_batches):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i % 10:
                print('%s: step %d, duration = %.3f' %
                      (datetime.now(), i- num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/- %.3f sec / batch' %
          (datetime.now(), info_string, num_batches, mn, sd))

In [12]:
batch_size = 32
height, width = 224,224
inputs = tf.random_uniform((batch_size,height,width,3))
with slim.arg_scope(resnet_arg_scope(is_training=False)):
    net, end_points = resnet_v2_152(inputs, 1000)

init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)
num_batches=100
time_tensorflow_run(sess, net, 'Forward',num_batches)

2018-01-24 10:16:39.856392: step 0, duration = 1.051
2018-01-24 10:16:50.357682: step 10, duration = 1.049
2018-01-24 10:17:00.853926: step 20, duration = 1.050
2018-01-24 10:17:11.369601: step 30, duration = 1.054
2018-01-24 10:17:21.884817: step 40, duration = 1.050
2018-01-24 10:17:32.395762: step 50, duration = 1.048
2018-01-24 10:17:42.889255: step 60, duration = 1.049
2018-01-24 10:17:53.381612: step 70, duration = 1.048
2018-01-24 10:18:03.878241: step 80, duration = 1.051
2018-01-24 10:18:14.373321: step 90, duration = 1.049
2018-01-24 10:18:23.821483: Forward across 100 steps, 1.050 +/- 0.002 sec / batch


In [13]:
import os
model_dir = 'resmodel'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
savepath = "./resmodel/model.ckpt"
saver = tf.train.Saver()
saver.save(sess, savepath)

'./resmodel/model.ckpt'

In [14]:
from tensorflow.python import pywrap_tensorflow  
checkpoint_path = os.path.join(model_dir, "model.ckpt")  
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)  
var_to_shape_map = reader.get_variable_to_shape_map()  
for key in var_to_shape_map:  
    print("tensor_name: ", key)  # Ouput variables name
    # print(reader.get_tensor(key)) # Output variables data

tensor_name:  resnet_v2_152/block2/unit_1/bottleneck_v2/preact/beta
tensor_name:  resnet_v2_152/block4/unit_2/bottleneck_v2/conv2/BatchNorm/moving_mean
tensor_name:  resnet_v2_152/postnorm/moving_variance
tensor_name:  resnet_v2_152/block3/unit_36/bottleneck_v2/preact/gamma
tensor_name:  resnet_v2_152/block3/unit_22/bottleneck_v2/preact/moving_mean
tensor_name:  resnet_v2_152/block3/unit_9/bottleneck_v2/conv2/BatchNorm/gamma
tensor_name:  resnet_v2_152/block3/unit_33/bottleneck_v2/conv3/biases
tensor_name:  resnet_v2_152/block3/unit_22/bottleneck_v2/conv2/BatchNorm/gamma
tensor_name:  resnet_v2_152/block2/unit_4/bottleneck_v2/conv2/weights
tensor_name:  resnet_v2_152/block3/unit_4/bottleneck_v2/preact/moving_mean
tensor_name:  resnet_v2_152/block3/unit_10/bottleneck_v2/conv1/BatchNorm/moving_mean
tensor_name:  resnet_v2_152/block4/unit_1/bottleneck_v2/preact/moving_mean
tensor_name:  resnet_v2_152/block3/unit_20/bottleneck_v2/conv2/BatchNorm/beta
tensor_name:  resnet_v2_152/block4/unit